# Part 1: Environment Setup

In [1]:
import os
os.getcwd()

'/Users/jingjietan/Desktop/CIMTAPR/dataset'

In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from IPython.display import display
import pickle

In [6]:
essays_path = "./raw/essays.csv"
mbti_path = "./raw/mbti.csv"

In [7]:
essays_psychofeature_path = "./psychofeature/essays_mairesse_labeled.csv"
mbti_psychofeature_path = "./psychofeature/mbti_mairesse_labeled.csv"

# Part 2: Load dataset

In [8]:
essays_dataset = pd.read_csv(essays_path,encoding='iso-8859-1')
mbti_dataset = pd.read_csv(mbti_path)

In [9]:
essays_psychofeature = pd.read_csv(essays_psychofeature_path)
mbti_psychofeature = pd.read_csv(mbti_psychofeature_path)

# Part 3: Organise dataset

In [11]:
essays_dataset.columns = ["user", "text", "E", "N", "A", "C", "O"]
essays_dataset['user'] = essays_dataset.index
for dim in ["O", "C","E", "A","N"]:
    essays_dataset[dim] = essays_dataset[dim].map({'y':'1','n':'0'})
for i,ed in essays_dataset.iterrows():
    essays_dataset.at[i,"ptype"] = str(int(str(ed["O"])+str(ed["C"])+str(ed["E"])+str(ed["A"])+str(ed["N"]),2))

essays_dataset = essays_dataset[["user", "O", "C","E", "A","N","ptype","text"]]

In [12]:
mbti_dataset.columns = ["user", "type","text"]
mbti_dataset['O'] = np.select(list(map(mbti_dataset['type'].str.contains, ['S','N'])), [0,1])
mbti_dataset['C'] = np.select(list(map(mbti_dataset['type'].str.contains, ['P','J'])), [0,1])
mbti_dataset['E'] = np.select(list(map(mbti_dataset['type'].str.contains, ['I','E'])), [0,1])
mbti_dataset['A'] = np.select(list(map(mbti_dataset['type'].str.contains, ['T','F'])), [0,1])
for i,ed in mbti_dataset.iterrows():
    mbti_dataset.at[i,"ptype"] = str(int(str(ed["O"])+str(ed["C"])+str(ed["E"])+str(ed["A"]),2))
mbti_dataset['user'] = mbti_dataset.index
mbti_dataset = mbti_dataset[["user", "O", "C","E", "A","ptype", "text"]]

# Part 4: Combine dataset

In [13]:
psychofeature_list = []
for i in range(len(essays_psychofeature)):
    psychofeature_list.append(np.array(essays_psychofeature.loc[i, :].values.tolist()[1:]))
essays_psychofeature_dataset = essays_dataset.assign(psychofeature = psychofeature_list)
essays_psychofeature_dataset

,user,O,C,E,A,N,ptype,text,psychofeature
0,0,1,0,0,1,1,19,"Well, right now I just woke up from a mid-day ...","[0.652286, -1.588505, -0.48468, -1.732113, -1...."
1,1,0,0,0,1,0,2,"Well, here we go with the stream of consciousn...","[1.213692, -1.281082, 0.94828, -1.079384, -0.4..."
2,2,1,1,0,0,1,25,An open keyboard and buttons to push. The thin...,"[-0.917103, -0.519972, 0.65841, -0.24954, -1.1..."
3,3,0,1,1,1,0,14,I can't believe it! It's really happening! M...,"[-0.348934, -0.025792, 1.28076, 0.111363, -0.3..."
4,4,1,0,1,1,0,22,"Well, here I go with the good old stream of co...","[1.569738, -1.464157, -0.625945, -1.166392, -1..."
...,...,...,...,...,...,...,...,...,...
2462,2462,0,1,0,0,1,9,I'm home. wanted to go to bed but remembe...,"[0.648704, 0.191965, -0.412708, 0.053884, 0.69..."
2463,2463,1,0,1,0,1,21,Stream of consiousnesssskdj. How do you s...,"[-0.233151, 0.829492, -1.317321, 1.031518, 0.7..."
2464,2464,0,0,0,1,0,2,"It is Wednesday, December 8th and a lot has be...","[-1.699756, -0.539734, 0.118524, -0.337989, -0..."
2465,2465,1,0,0,0,1,17,"Man this week has been hellish. Anyways, now i...","[-0.970174, 1.288341, 0.839773, 1.077985, 0.36..."


In [14]:
psychofeature_list = []
for i in range(len(mbti_psychofeature)):
    psychofeature_list.append(np.array(mbti_psychofeature.loc[i, :].values.tolist()[1:]))
mbti_psychofeature_dataset = mbti_dataset.assign(psychofeature =  psychofeature_list)
mbti_psychofeature_dataset

,user,O,C,E,A,ptype,text,psychofeature
0,0,1,1,0,1,13,'http://www.youtube.com/watch?v=qsXHcwe3krw|||...,"[0.08125, 0.857568, -1.245202, 1.351122, 0.154..."
1,1,1,0,1,0,10,'I'm finding the lack of me in these posts ver...,"[-0.227287, 0.860153, 0.681671, 0.581178, 0.95..."
2,2,1,0,0,0,8,'Good one _____ https://www.youtube.com/wat...,"[-0.647737, 0.073566, -0.60704, 0.069716, -0.1..."
3,3,1,1,0,0,12,"'Dear INTP, I enjoyed our conversation the o...","[0.916755, -0.891348, -1.059812, -0.843643, -0..."
4,4,1,1,1,0,14,'You're fired.|||That's another silly misconce...,"[0.714253, -1.007375, -1.040458, -0.77786, -0...."
...,...,...,...,...,...,...,...,...
8670,8670,0,0,0,1,1,'https://www.youtube.com/watch?v=t8edHB_h908||...,"[0.613203, -0.543976, 0.559328, -0.216531, -0...."
8671,8671,1,0,1,1,11,'So...if this thread already exists someplace ...,"[-0.473483, 0.082547, 1.187804, 0.294796, 0.42..."
8672,8672,1,0,0,0,8,'So many questions when i do these things. I ...,"[-0.030809, 0.388686, 0.984495, 0.281356, 0.16..."
8673,8673,1,0,0,1,9,'I am very conflicted right now when it comes ...,"[0.325698, 0.219271, 0.855555, -0.114935, 0.02..."


# Part 5: Dataset Spliting

In [15]:
from sklearn.model_selection import train_test_split
def split(dataset):
    return train_test_split(dataset, test_size=0.2, random_state=42, stratify=dataset.ptype)

essays_psychofeature_trainset,essays_psychofeature_testset = split(essays_psychofeature_dataset)
mbti_psychofeature_trainset,mbti_psychofeature_testset = split(mbti_psychofeature_dataset)

# Part 6: Text Feature Vectoriization

In [16]:
import re
def clear_text(data):
    cleaned_text=[]
    for sentence in data:
        sentence=sentence.lower()
        # removing links from text data
        sentence=re.sub('https?://[^\s<>"]+|www\.[^\s<>"]+',' ',sentence)
    
        # removing other symbols
        sentence=re.sub('[^0-9a-z]',' ',sentence)
        cleaned_text.append(sentence)
    return cleaned_text

In [18]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
nltk.download('wordnet')
nltk.download('omw-1.4') 

class Lemmatizer(object):
    def __init__(self):
        self.lemmatizer = WordNetLemmatizer()
    def __call__(self, sentence):
        return [self.lemmatizer.lemmatize(word) for word in sentence.split() if len(word)>2]

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/jingjietan/nltk_data...
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/jingjietan/nltk_data...


In [19]:
def expand(np_data):
    temp=[]
    for i in range(len(np_data)):
        temp.append(np.array(np_data[i]))
    return temp

In [20]:
essays_psychofeature_dataset.text = clear_text(essays_psychofeature_dataset.text)
mbti_psychofeature_dataset.text = clear_text(mbti_psychofeature_dataset.text)

#vectorise
vectorizer_essays=TfidfVectorizer(max_features=5000,stop_words='english',tokenizer=Lemmatizer())
vectorizer_essays.fit(essays_psychofeature_dataset.text)

vectorizer_mbti=TfidfVectorizer(max_features=5000,stop_words='english',tokenizer=Lemmatizer())
vectorizer_mbti.fit(mbti_psychofeature_dataset.text)

/Users/jingjietan/anaconda3/envs/phdwork/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/Users/jingjietan/anaconda3/envs/phdwork/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:408: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'wa'] not in stop_words.
  warnings.warn(


TfidfVectorizer(max_features=5000, stop_words='english',
                tokenizer=<__main__.Lemmatizer object at 0x11efc0310>)

In [21]:
essays_psychofeature_trainset.text=expand(vectorizer_essays.transform(essays_psychofeature_trainset.text).toarray())
essays_psychofeature_testset.text=expand(vectorizer_essays.transform(essays_psychofeature_testset.text).toarray())

In [22]:
mbti_psychofeature_trainset.text=expand(vectorizer_mbti.transform(mbti_psychofeature_trainset.text).toarray())
mbti_psychofeature_testset.text=expand(vectorizer_mbti.transform(mbti_psychofeature_testset.text).toarray())

# Part 6: Save single for experiment

In [ ]:
essays_psychofeature_otrainset = essays_psychofeature_trainset
mbti_psychofeature_otrainset = essays_psychofeature_trainset

essays_psychofeature_trainset,essays_psychofeature_validationset = split(essays_psychofeature_otrainset)
mbti_psychofeature_trainset,mbti_psychofeature_validationset = split(mbti_psychofeature_otrainset)

In [26]:
save_name = "essays"
pickle.dump(essays_psychofeature_trainset, open("./merged/essays_train.pickle", 'wb'))
pickle.dump(essays_psychofeature_validationset, open("./merged/essays_validation.pickle", 'wb'))
pickle.dump(essays_psychofeature_testset, open("./merged/essays_test.pickle", 'wb'))

save_name = "mbti"
pickle.dump(mbti_psychofeature_trainset, open("./merged/mbti_train.pickle", 'wb'))
pickle.dump(mbti_psychofeature_validationset, open("./merged/mbti_validation.pickle", 'wb'))
pickle.dump(mbti_psychofeature_testset, open("./merged/mbti_test.pickle", 'wb'))